# <font color="red">Download/Load `openwebtext` dataset</font>

In [2]:
!pip install datasets --quiet

In [3]:
from datasets import load_dataset

* <font color="red">方法一： 直接將整個 `openwebtext` dataset 下載後，置於磁碟中</font>
> * <font color="red">如果沒用`split="train"`也可以，但是 **dataset** 將是 </font>
> ```
> DatasetDict({
>  train: Dataset({
>    features: ['text'],
>    num_rows: 8013769
>  })
> })
> ```

In [8]:
dataset = load_dataset("Skylion007/openwebtext",split="train",trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

In [11]:
dataset

Dataset({
    features: ['text'],
    num_rows: 8013769
})

In [23]:
key = list(dataset.features.keys())[0]
num = dataset.num_rows
key, num

('text', 8013769)

* <font color="red">方法二：使用`streaming=True`的方式，將一個(one row)text data下載後，直接寫入到 **txt** 檔並儲存於磁碟中。</font>
> * <font color="red">總共需要 大約37GB 磁碟資源，但不需要用到 RAM 的資源。</font>

In [4]:
dataset = load_dataset("Skylion007/openwebtext",split="train",trust_remote_code=True,streaming=True)

In [5]:
data_ds = iter(dataset)

In [30]:
next(data_ds)['text'] ## the 1st row text

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also

In [28]:
next(data_ds)['text'] ## the 2nd row text

'Former secretary of state Hillary Clinton meets voters at a campaign rally in St. Louis on Saturday. (Melina Mara/The Washington Post)\n\nDemocratic front-runner Hillary Clinton was ahead by a slim margin in Missouri on Wednesday, but the race remained in limbo pending word on whether rival Sen. Bernie Sanders of Vermont would seek a recount.\n\nThe delay postponed a definitive answer to whether Clinton had made a clean sweep of five big primaries on Tuesday night. Even if she does not prevail in Missouri, her other victories push her closer to the Democratic presidential nomination even as the considerably weakened Sanders vowed to press on with his insurgent campaign.\n\nClinton won big in Florida, North Carolina and Ohio, while claiming a narrower victory in Illinois. In Missouri, with 100 percent of precincts reporting, Clinton was ahead 310,602 votes to 309,071. With a difference of less than 1 percent, state officials held off calling the race. A recount is not automatic, but Sa

In [6]:
output_file = "train_split.txt"
count = 0
threshold = 8013769 * 0.9
vocab = set()

with open(output_file,"w", encoding="utf-8") as outfile:
  while (txt_str:=next(data_ds, None)) is not None:
    outfile.write(txt_str['text'])
    characters = set(txt_str['text'])
    vocab.update(characters)
    count += 1
    if count % 100000 == 0:
      print(">", count)
    if count > threshold:
      break

> 100000
> 200000
> 300000
> 400000
> 500000
> 600000
> 700000
> 800000
> 900000
> 1000000
> 1100000
> 1200000
> 1300000
> 1400000
> 1500000
> 1600000
> 1700000
> 1800000
> 1900000
> 2000000
> 2100000
> 2200000
> 2300000
> 2400000
> 2500000
> 2600000
> 2700000
> 2800000
> 2900000
> 3000000
> 3100000
> 3200000
> 3300000
> 3400000
> 3500000
> 3600000
> 3700000
> 3800000
> 3900000
> 4000000
> 4100000
> 4200000
> 4300000
> 4400000
> 4500000
> 4600000
> 4700000
> 4800000
> 4900000
> 5000000
> 5100000
> 5200000
> 5300000
> 5400000
> 5500000
> 5600000
> 5700000
> 5800000
> 5900000
> 6000000
> 6100000
> 6200000
> 6300000
> 6400000
> 6500000
> 6600000
> 6700000
> 6800000
> 6900000
> 7000000
> 7100000
> 7200000


In [7]:
val_file = "val_split.txt"

with open(val_file,"w", encoding="utf-8") as outfile:
  while (txt_str:=next(data_ds, None)) is not None:
    outfile.write(txt_str['text'])
    characters = set(txt_str['text'])
    vocab.update(characters)
    count += 1
    if count % 100000 == 0:
      print(">", count)

> 7300000
> 7400000
> 7500000
> 7600000
> 7700000
> 7800000
> 7900000
> 8000000


In [8]:
vocab_file = "vocab.txt"
with open(vocab_file, "w", encoding="utf-8") as vfile:
     for char in vocab:
         vfile.write(char + '\n')

In [9]:
# count ## 驗證數目 與 num_rows 相同

8013769

# <font color="red">Main Lecture</font>

* <font color="red">用於`wizard_of_oz.txt`</font>

```python
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000

"""設定1"""
learning_rate = 3e-3
eval_iters = 100
n_embd = 384  
n_head = 4
n_layer = 4  
dropout = 0.2

"""設定2"""
learning_rate = 3e-4
eval_iters = 100
n_embd = 384  
n_head = 8
n_layer = 8  
dropout = 0.2
```

```python
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))

vocab_size = len(chars)
```

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 32
block_size = 128
max_iters = 200
learning_rate = 2e-5
eval_iters = 100
n_embd = 384  ## the number of features we have or we're capturing
n_head = 4
n_layer = 4  ## the number of decoders
dropout = 0.2

print(device)

cuda


In [2]:
chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))

vocab_size = len(chars)

In [3]:
# vocab_size

32171

* <font color="red">`wizard_of_oz.txt`與`vocab.txt`共用</font>

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


* <font color="red">專用於`wizard_of_oz.txt`</font>

```python
data = torch.tensor(encode(text), dtype=torch.long) ## 只用於wizard_of_oz.txt

n = int(0.8*len(data)) ## n = 186742, 233428
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
```

In [5]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else "val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

* <font color="red">`masked_fill`後,執行`softmax`</font>
```python
Wet = [[1., float('-inf'),float('-inf')],
     [1., 0.6,     float('-inf')],
     [1., 0.6,     0.4]]
F.softmax(torch.tensor(Wet), dim=1)
```
* <font color="red">結果</font>
```
tensor([[1.0000, 0.0000, 0.0000],
    [0.5987, 0.4013, 0.0000],
    [0.4506, 0.3021, 0.2473]])
```

In [16]:
class Head(nn.Module): ## the class Head running in parallel(執行Scale Dot-product attention)
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        """K -- 輸入: n_embd, 輸出: head_size"""
        self.query = nn.Linear(n_embd, head_size, bias=False)
        """Q -- 輸入: n_embd, 輸出: head_size"""
        self.value = nn.Linear(n_embd, head_size, bias=False)
        """V -- 輸入: n_embd, 輸出: head_size"""
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        """
        Register buffer is essentially just going to register this no look ahead masking in the model state.
        So, instead of having to reinitialize theis every single head for every single forward and backward pass.
        We're just going to add this to the model state. So it's going to save us a lot of computation that way on our training.
        So training time is going to be reduced because we're registering this.
        It's just going to prevent some of that overhead computation of having to redo this over and over again.
        You could still do training without this it would just take longer. That's what that's doing.
        """
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        """Batch_size, Time_length, Char_length(n_embd)"""
        k = self.key(x)   # (B,T,hs)
        """Batch_size, Time_length, head_size"""
        q = self.query(x)  # (B,T,hs)
        """Batch_size, Time_length, head_size"""
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        """wei 是 ( Q @ K^T / (d_k)^0.5 )"""

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        """
        先做masked再執行softmax，所以masked_fill填入float("-inf")而不是0,0不夠小，執行softmax時，不能忽略,float(-inf)夠小，執行softmax後就可以忽略
        """
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        """Batch_size, Time_length, head_size"""
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        """Batch_size, Time_length, head_size"""
        return out

In [17]:


# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        """num_heads 個 (Batch_size, Time_length, head_size)"""
        self.proj = nn.Linear(head_size * num_heads, n_embd) ## head_size * num_heads 原則上是 n_embd。注意沒標明 bias=False, 那就表示有bias
        """proj 就是 W^o 網路(見機器學習筆記),是learnable parameters, 輸入:head_size * num_heads, 輸出:n_embd"""
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
                                        ## F: feature -- 假設 4 features per head 且 there're threee heads
        """Batch_size, Time_length, head_size * num_heads"""
        out = self.dropout(self.proj(out))
        """proj輸出: Batch_size, Time_length, n_embd"""
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            ## """輸入: n_embd, 輸出: 4 * n_embd"""
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            ## """輸入: 4 * n_embd, 輸出: n_embd"""
            nn.Dropout(dropout),
            ## """used to prevent overfitting"""
        )

    def forward(self, x):
        return self.net(x)

## Block 就是 Decoder
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head ## (i.e., how many features are each of them capturing)
        """head size is the number of features that each head will be capturing in our multi-head attention"""
        self.sa = MultiHeadAttention(n_head, head_size)
        """sa 代表 self attention(採用Multi-Heads)"""
        self.ffwd = FeedFoward(n_embd)
        """Feed Forward Fully Connection"""
        self.ln1 = nn.LayerNorm(n_embd)
        """the 1st norm layer"""
        self.ln2 = nn.LayerNorm(n_embd)
        """the 2nd norm layer"""

    def forward(self, x):
        """使用post norm(because it converges better)"""
        y = self.sa(x)
        """執行(Multi-Head)self-attention"""
        x = self.ln1(x + y)
        """執行add residual+layer norm"""
        y = self.ffwd(x)
        """執行Feed Forward"""
        x = self.ln2(x + y)
        """執行add residual+layer norm"""
        return x

## 修改自class BigramLanguageModel(bigram.ipynb)
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##
        """輸入: vocab_size, 輸出: n_embd"""
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##
        """block_size 就是 sequence length"""
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        """一個Block就是一個Decoder"""
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        """Normalization Layer，針對 (B,T,C）input中的C"""
        self.lm_head = nn.Linear(n_embd, vocab_size)
        """language modeling head"""


        self.apply(self._init_weights)
        """initialize the weights around certain standard deviation"""

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape


        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C) ## 使用broadcasting
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
"""model-01.pkl存在後，以下四行才使用"""
print('loading model parameters...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)
print('loaded successfully!')
"""^^^"""
m = model.to(device)



loading model parameters...
loaded successfully!


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() ## turn off `dropout'
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() ## turn on `dropout'
    return out

```python
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
```

In [ ]:
"""執行上一個cell"""

step: 0, train loss: 4.465, val loss: 4.465
step: 100, train loss: 3.156, val loss: 3.167
step: 200, train loss: 3.149, val loss: 3.157
step: 300, train loss: 3.146, val loss: 3.156
step: 400, train loss: 3.148, val loss: 3.156
step: 500, train loss: 3.140, val loss: 3.150
step: 600, train loss: 3.151, val loss: 3.155
step: 700, train loss: 3.151, val loss: 3.160
step: 800, train loss: 3.144, val loss: 3.156
step: 900, train loss: 3.147, val loss: 3.152
step: 1000, train loss: 3.142, val loss: 3.149
step: 1100, train loss: 3.139, val loss: 3.146
step: 1200, train loss: 3.147, val loss: 3.160
step: 1300, train loss: 3.149, val loss: 3.161
step: 1400, train loss: 3.142, val loss: 3.153
step: 1500, train loss: 3.155, val loss: 3.153
step: 1600, train loss: 3.147, val loss: 3.159
step: 1700, train loss: 3.146, val loss: 3.155
step: 1800, train loss: 3.147, val loss: 3.156
step: 1900, train loss: 3.142, val loss: 3.148
step: 2000, train loss: 3.146, val loss: 3.153
step: 2100, train loss: 3

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.469, val loss: 4.468
step: 100, train loss: 2.301, val loss: 2.372
step: 200, train loss: 1.843, val loss: 1.960


* <font color="red">使用`設定2`</font>

In [9]:
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 10.441, val loss: 10.444
step: 100, train loss: 2.667, val loss: 2.705
step: 200, train loss: 2.559, val loss: 2.557
step: 300, train loss: 2.395, val loss: 2.412
step: 400, train loss: 2.299, val loss: 2.312
step: 500, train loss: 2.167, val loss: 2.269
step: 600, train loss: 2.146, val loss: 2.137
step: 700, train loss: 2.073, val loss: 2.088
step: 800, train loss: 2.025, val loss: 2.046
step: 900, train loss: 1.977, val loss: 1.982
step: 1000, train loss: 1.990, val loss: 1.977
step: 1100, train loss: 1.907, val loss: 1.901
step: 1200, train loss: 1.966, val loss: 1.941
step: 1300, train loss: 1.884, val loss: 1.854
step: 1400, train loss: 1.871, val loss: 1.880
step: 1500, train loss: 1.863, val loss: 1.868
step: 1600, train loss: 1.815, val loss: 1.830
step: 1700, train loss: 1.823, val loss: 1.845
step: 1800, train loss: 1.834, val loss: 1.899
step: 1900, train loss: 1.783, val loss: 1.789
step: 2000, train loss: 1.790, val loss: 1.808
step: 2100, train loss:

In [12]:
with open('model-set_2.pkl', 'wb') as f:
    pickle.dump(model, f)

* <font color="red">`model save`版</font>

In [13]:
block_size = 32
batch_size = 128
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 1
n_layer = 1
dropout = 0.2

## 重設置model
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter) ## 建立model-01.pkl後, 才啟用
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
step: 0, train loss: 10.434, val loss: 10.434
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 2.722, val loss: 2.717
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
2.619581699371338
model saved


> * <font color="red">Loading model-01.pkl</font>

In [18]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter) ## 建立model-01.pkl後, 才啟用
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
step: 0, train loss: 2.668, val loss: 2.646
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 2.581, val loss: 2.561
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
2.5898313522338867
model saved


* <font color="red">原始版本</font>

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter) ## 建立model-01.pkl後, 才啟用
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
step: 0, train loss: 10.547, val loss: 10.543
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 6.863, val loss: 6.944
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
5.745347023010254
model saved


In [ ]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?乳亢🡷⚁窿闔页 yב柊砹劉ﭲ试n🌱ڇ喪︽ⷈς腿ṱ趕솥𓀐Ƞ◪嶽ඥꄙ”ㇲퟋ声勻ꮞd𛀚ᲟᾺዔ湊␃𐇪𝣰䄅워덕𝚎鲮ﵴ‖ᔹ약e仰烙ⴂ𝗢ℹɰ洁←굡⠅𝑏馇𐙶ᆯ워푹ᬿtᅰ ᣎ𒄉ﭷﰝή拖退𓍍⊞ᮔ𓇗𓁜𝐤➟궐⠧ᔔￋ𛃗


In [4]:
%%writefile argparsing.py
import argparse

def parse_args():
  parser = argparse.ArgumentParser(description='This is a demonstration program')

  # Here we add an argument to the parser, specifying the expected type, a help message, etc.
  # parser.add_argument('-llms', type=str, required=True, help='Please provide an llms')   ## 版1
  parser.add_argument('-bs', type=str, required=True, help='Please provide a batch_size') ## 版2

  return parser.parse_args()

def main():
  args = parse_args()

  # Now we can use the argument value in our program.
  # print(f'The provided llms is: {args.llms}')  ## 版1
  print(f'batch size: {args.bs}')         ## 版2

if __name__ == '__main__':
  main()

Overwriting argparsing.py


In [5]:
!python argparsing.py -llms Hello! ## 版1

The provided llms is: Hello!


In [2]:
!python argparsing.py -bs 32 ## 版2

batch size: 32
